In [3]:
!git clone https://github.com/l3vels/L3AGI.git

fatal: destination path 'L3AGI' already exists and is not an empty directory.


In [8]:
%%writefile /content/L3AGI/apps/server/test.py
from xagent import XAgent, XAgentClient, XAgentConfig, XAgentEvalConfig
from xagent.tools import XSerpGoogleSearch
from langsmith import Client

# Refactored function to initialize the XAgent
def xagent_factory():
    # Initialize the XAgent with necessary configurations
    xagent = XAgent(
        model_name="gpt-3.5-turbo",
        temperature=0,
        tools=[XSerpGoogleSearch()],
        system_message="Your system message here",
        output_parser="Your output parser here",
        max_iterations=5,
        verbose=True,
        handle_parsing_errors="Check your output and make sure it conforms!"
    )
    return xagent

# Initialize the agent
agent = xagent_factory()

# Initialize the Langsmith client (assuming it can interact with XAgent)
client = Client()

# Evaluation configuration using XAgent's configuration class
eval_config = XAgentEvalConfig(
    evaluators=[
        "qa",
        XAgentEvalConfig.Criteria("helpfulness"),
        XAgentEvalConfig.Criteria("conciseness"),
    ],
    input_key="input",
    eval_llm=XAgent(model_name="gpt-3.5-turbo", temperature=0.5),
)

# Running evaluation on the dataset with XAgent
chain_results = client.run_on_dataset(
    dataset_name="test-dataset",
    llm_or_chain_factory=xagent_factory,
    evaluation=eval_config,
    concurrency_level=1,
    verbose=True,
)

Overwriting /content/L3AGI/apps/server/test.py


In [17]:
%%writefile /content/L3AGI/apps/server/agents/conversational/conversational.py
import asyncio

from xagent import XAgent, XAgentExecutor, XAgentConfig, XAgentCallbackHandler, XAgentErrorHandling
from xagent.tools import XSerpGoogleSearch, speech_to_text, text_to_speech
from xagent.memory import XAgentMemory
from xagent.output_parser import XAgentOutputParser
from xagent.streaming_aiter import XAgentAsyncCallbackHandler
from config import Config
from memory.zep.zep_memory import ZepMemory
from postgres import PostgresChatMessageHistory
from services.pubsub import ChatPubSubService
from services.run_log import RunLogsManager
from typings.agent import AgentWithConfigsOutput
from typings.config import AccountSettings, AccountVoiceSettings
from utils.model import get_llm
from utils.system_message import SystemMessageBuilder

class ConversationalAgent(BaseAgent):
    async def run(
        self,
        settings: AccountSettings,
        voice_settings: AccountVoiceSettings,
        chat_pubsub_service: ChatPubSubService,
        agent_with_configs: AgentWithConfigsOutput,
        tools,
        prompt: str,
        voice_url: str,
        history: PostgresChatMessageHistory,
        human_message_id: str,
        run_logs_manager: RunLogsManager,
        pre_retrieved_context: str,
    ):
        memory = XAgentMemory(
            session_id=str(self.session_id),
            url=Config.ZEP_API_URL,
            api_key=Config.ZEP_API_KEY,
            memory_key="chat_history",
            return_messages=True,
        )

        memory.human_name = self.sender_name
        memory.ai_name = agent_with_configs.agent.name

        system_message = SystemMessageBuilder(
            agent_with_configs, pre_retrieved_context
        ).build()

        res: str

        try:
            if voice_url:
                configs = agent_with_configs.configs
                prompt = speech_to_text(voice_url, configs, voice_settings)

            llm = get_llm(
                settings,
                agent_with_configs,
            )

            streaming_handler = XAgentAsyncCallbackHandler()

            llm.streaming = True
            # llm.callbacks = [
            #     run_logs_manager.get_agent_callback_handler(),
            #     streaming_handler,
            # ]

            # agent = initialize_agent(
            #     tools,
            #     llm,
            #     agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            #     verbose=True,
            #     memory=memory,
            #     handle_parsing_errors="Check your output and make sure it conforms!",
            #     agent_kwargs={
            #         "system_message": system_message,
            #         "output_parser": XAgentOutputParser(),
            #     },
            #     callbacks=[run_logs_manager.get_agent_callback_handler()],
            # )

            agent = XAgent(
                model_name="gpt-3.5-turbo",
                temperature=0,
                tools=tools,
                system_message=system_message,
                output_parser=XAgentOutputParser(),
                max_iterations=5,
                verbose=True,
                handle_parsing_errors="Check your output and make sure it conforms!",
            )

            agent_executor = XAgentExecutor(agent=agent, tools=tools, verbose=True)

            chunks = []
            final_answer_detected = False

            async for event in agent_executor.astream_events(
                {"input": prompt}, version="v1"
            ):
                kind = event["event"]

                if kind == "on_chat_model_stream":
                    content = event["data"]["chunk"].content
                    if content:
                        chunks.append(content)
                        # Check if the last three elements in chunks,
                        # when stripped, are "Final", "Answer", and ":"
                        if (
                            len(chunks) >= 3
                            and chunks[-3].strip() == "Final"
                            and chunks[-2].strip() == "Answer"
                            and chunks[-1].strip() == ":"
                        ):
                            final_answer_detected = True
                            continue

                        if final_answer_detected:
                            yield content

            full_response = "".join(chunks)
            final_answer_index = full_response.find("Final Answer:")
            if final_answer_index != -1:
                # Add the length of the phrase "Final Answer:"
                # and any subsequent whitespace or characters you want to skip
                start_index = final_answer_index + len("Final Answer:")
                # Optionally strip leading whitespace
                res = full_response[start_index:].lstrip()
            else:
                res = "Final Answer not found in response."

        except Exception as err:
            res = XAgentErrorHandling.handle_agent_error(err)

            memory.save_context(
                {
                    "input": prompt,
                    "chat_history": memory.load_memory_variables({})["chat_history"],
                },
                {
                    "output": res,
                },
            )

            yield res

        try:
            configs = agent_with_configs.configs
            voice_url = None
            if "Voice" in configs.response_mode:
                voice_url = text_to_speech(res, configs, voice_settings)
                pass
        except Exception as err:
            res = f"{res}\n\n{XAgentErrorHandling.handle_agent_error(err)}"

            yield res

        ai_message = history.create_ai_message(
            res,
            human_message_id,
            agent_with_configs.agent.id,
            voice_url,
        )

        chat_pubsub_service.send_chat_message(chat_message=ai_message)




Overwriting /content/L3AGI/apps/server/agents/conversational/conversational.py


In [19]:
%%writefile /content/L3AGI/apps/server/agents/agent_simulations/agent/dialogue_agent_with_tools.py
from typing import List, Optional

from xagent import XAgent, XAgentExecutor, XAgentConfig, XAgentCallbackHandler, XAgentErrorHandling
from xagent.memory import XAgentMemory
from xagent.output_parser import XAgentOutputParser
from config import Config
from services.run_log import RunLogsManager
from typings.agent import AgentWithConfigsOutput


class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        agent_with_configs: AgentWithConfigsOutput,
        system_message: SystemMessage,
        model: XAgent,  # Replace ChatOpenAI with XAgent
        tools: List[any],
        session_id: str,
        sender_name: str,
        is_memory: bool = False,
        run_logs_manager: Optional[RunLogsManager] = None,
        **tool_kwargs,
    ) -> None:
        super().__init__(name, agent_with_configs, system_message, model)
        self.tools = tools
        self.session_id = session_id
        self.sender_name = sender_name
        self.is_memory = is_memory
        self.run_logs_manager = run_logs_manager

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """

        memory: XAgentMemory

        # FIXME: This is a hack to get the memory working
        if self.is_memory:
            memory = XAgentMemory(
                session_id=self.session_id,
                url=Config.ZEP_API_URL,
                api_key=Config.ZEP_API_KEY,
                memory_key="chat_history",
                return_messages=True,
            )

            memory.human_name = self.sender_name
            memory.ai_name = self.agent_with_configs.agent.name
            memory.auto_save = False
        # else:
        #     memory = ConversationBufferMemory(
        #         memory_key="chat_history", return_messages=True
        #     )

        callbacks = []

        if self.run_logs_manager:
            self.model.callbacks = [self.run_logs_manager.get_agent_callback_handler()]
            callbacks.append(self.run_logs_manager.get_agent_callback_handler())

        agent = XAgent(
            model_name="gpt-3.5-turbo",
            temperature=0,
            tools=self.tools,
            system_message=self.system_message.content,
            output_parser=XAgentOutputParser(),
            max_iterations=5,
            verbose=True,
            handle_parsing_errors="Check your output and make sure it conforms!",
            memory=memory,
            callbacks=callbacks,
        )

        agent_executor = XAgentExecutor(agent=agent, tools=self.tools, verbose=True)

        prompt = "\n".join(self.message_history + [self.prefix])

        res = agent_executor.run(input=prompt)

        # FIXME: is memory
        # memory.save_ai_message(res)

        message = AIMessage(content=res)

        return message.content


Overwriting /content/L3AGI/apps/server/agents/agent_simulations/agent/dialogue_agent_with_tools.py
